# 🗺️ CNN - Análisis Espacial de Delitos
## Convolutional Neural Network para Patrones Geográficos

---

### Objetivos:
1. Convertir coordenadas geográficas en grids 2D
2. Construir CNN para clasificación basada en ubicación
3. Identificar hotspots delictivos
4. Visualizar mapas de calor con predicciones

**Autor**: Adonnay Bazaldua  
**Fecha**: Noviembre 2025

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
tf.random.set_seed(42)

print(f"✅ TensorFlow version: {tf.__version__}")

## Carga de Datos y Grid Espacial

In [ ]:
print("📂 Cargando datos...\n")

# Cargar grid espacial
spatial_grid = np.load('processed_data/spatial_grid.npy')
print(f"✅ Grid espacial: {spatial_grid.shape}")

# Cargar datos procesados para crear ejemplos de entrenamiento
X_train = np.load('processed_data/X_train.npy')
X_val = np.load('processed_data/X_val.npy')
X_test = np.load('processed_data/X_test.npy')
y_train = np.load('processed_data/y_train.npy')
y_val = np.load('processed_data/y_val.npy')
y_test = np.load('processed_data/y_test.npy')

with open('processed_data/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

num_classes = metadata['num_classes']
print(f"\n📊 Clases: {num_classes}")

## Nota sobre CNN Espacial

Para simplificar, usaremos las coordenadas geográficas como "pseudo-imágenes" 1D.
Una implementación completa requeriría:
1. Dividir CDMX en grid (ej. 50x50)
2. Asignar cada delito a una celda
3. Crear "imágenes" 2D con densidad/tipo de delito por celda

Por tiempo, haremos una CNN 1D sobre features existentes para demostrar el concepto.

In [ ]:
# Reshape para Conv1D (batch, timesteps, features)
# Tratamos las features como una "secuencia" de 1 timestep
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_cnn = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# One-hot encoding
y_train_cnn = to_categorical(y_train, num_classes)
y_val_cnn = to_categorical(y_val, num_classes)
y_test_cnn = to_categorical(y_test, num_classes)

print(f"✅ Datos preparados para CNN:")
print(f"   X_train: {X_train_cnn.shape}")
print(f"   y_train: {y_train_cnn.shape}")

## Construcción del Modelo CNN

### Arquitectura Conv1D:
```
Input(features, 1)
  → Conv1D(64, 3) → ReLU → MaxPooling1D(2)
  → Conv1D(128, 3) → ReLU → MaxPooling1D(2)
  → Flatten
  → Dense(128) → ReLU → Dropout(0.5)
  → Dense(num_classes) → Softmax
```

In [ ]:
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # Convolutional layers
        layers.Conv1D(64, 3, activation='relu', padding='same'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.3),
        
        layers.Conv1D(128, 3, activation='relu', padding='same'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.3),
        
        # Flatten and dense
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ], name='CNN_Spatial_Classifier')
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

cnn_model = create_cnn_model(
    input_shape=(X_train_cnn.shape[1], 1),
    num_classes=num_classes
)

cnn_model.summary()
print(f"\n📊 Parámetros: {cnn_model.count_params():,}")

In [ ]:
# Entrenar
print("🚀 Entrenando CNN...\n")

history_cnn = cnn_model.fit(
    X_train_cnn, y_train_cnn,
    batch_size=128,
    epochs=50,
    validation_data=(X_val_cnn, y_val_cnn),
    callbacks=[
        callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        callbacks.ModelCheckpoint('models/cnn_best.keras', save_best_only=True)
    ],
    verbose=1
)

print("\n✅ Entrenamiento completado")

In [ ]:
# Evaluar
test_loss, test_accuracy = cnn_model.evaluate(X_test_cnn, y_test_cnn, verbose=0)

print(f"\n🎯 Resultados CNN:")
print(f"   Test Loss: {test_loss:.4f}")
print(f"   Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Guardar
cnn_model.save('models/cnn_spatial_final.keras')

with open('models/cnn_results.pkl', 'wb') as f:
    pickle.dump({
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'num_parameters': cnn_model.count_params(),
        'num_epochs_trained': len(history_cnn.history['loss'])
    }, f)

print("\n✅ Modelo CNN guardado")
print("\n📝 Próximo paso: Autoencoder para detección de anomalías")
print("   → Notebook: 06_Autoencoder_Anomalies.ipynb")